In [56]:
using JuMP
using GLPK

In [57]:
m = Model();
set_optimizer(m, GLPK.Optimizer);

In [58]:
arestas = [(1,2,4),(1,3,1),(1,4,1),(2,3,1),(2,4,1),(2,5,1)]
K = collect(1:4)
V = collect(1:5)

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [59]:
function create_arestas_k(arestas)
    arestas_k = [[],[],[],[]]
    for a in arestas
        push!(arestas_k[a[3]], (a))
    end
    return arestas_k
end


create_arestas_k (generic function with 1 method)

In [60]:
arestas_k = create_arestas_k(arestas)

4-element Vector{Vector{Any}}:
 [(1, 3, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1)]
 []
 []
 [(1, 2, 4)]

In [61]:
@variable(m, x[i in arestas], Bin)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [(1, 2, 4), (1, 3, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1)]
And data, a 6-element Vector{VariableRef}:
 x[(1, 2, 4)]
 x[(1, 3, 1)]
 x[(1, 4, 1)]
 x[(2, 3, 1)]
 x[(2, 4, 1)]
 x[(2, 5, 1)]

In [62]:
@constraint(m, [k in K], sum(x[i] for i in arestas_k[k]) <=1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[(1, 3, 1)] + x[(1, 4, 1)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)] ≤ 1.0
 0 ≤ 1.0
 0 ≤ 1.0
 x[(1, 2, 4)] ≤ 1.0

In [63]:
function create_arestas_v(arestas)
    arestas_v = [[],[],[],[],[]]
    for a in arestas
        push!(arestas_v[a[1]], (a))
        push!(arestas_v[a[2]], (a))
    end
    return arestas_v
end

create_arestas_v (generic function with 1 method)

In [64]:
arestas_v = create_arestas_v(arestas)

5-element Vector{Vector{Any}}:
 [(1, 2, 4), (1, 3, 1), (1, 4, 1)]
 [(1, 2, 4), (2, 3, 1), (2, 4, 1), (2, 5, 1)]
 [(1, 3, 1), (2, 3, 1)]
 [(1, 4, 1), (2, 4, 1)]
 [(2, 5, 1)]

In [65]:
@constraint(m, [v in V], sum(x[i] for i in arestas_v[v]) <=1)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5]
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[(1, 2, 4)] + x[(1, 3, 1)] + x[(1, 4, 1)] ≤ 1.0
 x[(1, 2, 4)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)] ≤ 1.0
 x[(1, 3, 1)] + x[(2, 3, 1)] ≤ 1.0
 x[(1, 4, 1)] + x[(2, 4, 1)] ≤ 1.0
 x[(2, 5, 1)] ≤ 1.0

In [66]:
@objective(m, Max, sum(x[i] for i in arestas))

x[(1, 2, 4)] + x[(1, 3, 1)] + x[(1, 4, 1)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)]

In [67]:
println(m)

Max x[(1, 2, 4)] + x[(1, 3, 1)] + x[(1, 4, 1)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)]
Subject to
 x[(1, 3, 1)] + x[(1, 4, 1)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)] ≤ 1.0
 0 ≤ 1.0
 0 ≤ 1.0
 x[(1, 2, 4)] ≤ 1.0
 x[(1, 2, 4)] + x[(1, 3, 1)] + x[(1, 4, 1)] ≤ 1.0
 x[(1, 2, 4)] + x[(2, 3, 1)] + x[(2, 4, 1)] + x[(2, 5, 1)] ≤ 1.0
 x[(1, 3, 1)] + x[(2, 3, 1)] ≤ 1.0
 x[(1, 4, 1)] + x[(2, 4, 1)] ≤ 1.0
 x[(2, 5, 1)] ≤ 1.0
 x[(1, 2, 4)] binary
 x[(1, 3, 1)] binary
 x[(1, 4, 1)] binary
 x[(2, 3, 1)] binary
 x[(2, 4, 1)] binary
 x[(2, 5, 1)] binary



In [68]:
optimize!(m)

In [69]:
objective_value(m)

1.0

In [70]:
@show value.(x)

value.(x) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [(1, 2, 4), (1, 3, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1)]
And data, a 6-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [(1, 2, 4), (1, 3, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1)]
And data, a 6-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0